In [3]:
import pandas as pd

Import clean data

In [4]:
good_bad = pd.read_csv("good_bad_clients.csv")
application_records = pd.read_csv("application_record_clean.csv")
credit_records = pd.read_csv("credit_record_clean.csv")

In [20]:
def month_weight(month, number_of_records, mnor, historical_range):
    
    """
    Funkcja przyporządkowująca wagę danemu miesiącu (month) z uwzględnieniem liczby wszystkich miesięcy (number_of_months).
    Wagi maleją w sposób liniowy, a ich suma daje 1.
    """
    
    k = abs(month) + 1
    n = number_of_records
    
    if n < mnor:
        weight = 0
    elif mnor <= n <= historical_range:
        weight = -2*k/(n*(n+1))+2/n
    else:
        n = historical_range
        if k <= historical_range:
            weight = -2*k/(n*(n+1))+2/n
        else:
            weight = 0
              
                
    return weight

In [18]:
def good_bad_function(ID, status, threshold):
    
    """
    Funkcja określająca czy dany klient może otrzymać kredyt.
    """
    
    if status >= threshold:
        judgment = 1
    else:
        judgment = 0
    
    
    return judgment

In [11]:
def account_activity(ID, last_record):
    
    """
    Funkcja zwracająca obecny stan konta klienta.
    Konto aktywne - 1
    Konto nieaktywne - 0
    """
        
    if last_record == 0:
        active = 1
    else:
        active = 0
        
        
    return active

In [19]:
def status2number(month, status, number_of_records, mnor, historical_range):
    
    """
    W oparciu o funkcję month_weight, status2number przekształca status z wybranego miesiąca na liczbę z przedziału <-1,1>.
    
    """
    
    if month <= historical_range-1:
        weight = month_weight(month,
                              number_of_records,
                              mnor,
                              historical_range)

        if status is "5":
            value = -1
        elif status is "X":
            value = 0
        elif status is "C":
            value = 1
        else:
            n = int(status)
            value = 1/2**(n+1)-1
    else:
        weight = 1
        value = 0
    
    
    return weight * value

<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-19-9579a0827f27>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if status is "5":
<ipython-input-19-9579a0827f27>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif status is "X":
<ipython-input-19-9579a0827f27>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif status is "C":


In [14]:
def good_bad_function(ID, status, threshold):
    
    """
    Funkcja określająca czy dany klient może otrzymać kredyt.
    """
    
    if status >= threshold:
        judgment = 1
    else:
        judgment = 0
    
    
    return judgment

Function to create date frame with selecterd threshold

In [16]:
def main(data_frame, minimum_number_of_records=4, historical_range=24, threshold=0):
    
    """
    text
    text
    
    """
    
    data_frame["MONTHS_BALANCE"] = data_frame["MONTHS_BALANCE"].abs()
    
    extra_frame = data_frame.groupby(by="ID")["MONTHS_BALANCE"].min().reset_index(name="LAST_RECORD")
    data_frame = pd.merge(data_frame, extra_frame, on="ID", how="left")
    
    extra_frame = data_frame.groupby(by="ID")["MONTHS_BALANCE"].count().reset_index(name="NUMBER_OF_RECORDS")
    data_frame = pd.merge(data_frame, extra_frame, on="ID", how="left")
    
    data_frame["MONTHS"] = data_frame["MONTHS_BALANCE"] - data_frame["LAST_RECORD"]
    

    data_frame["ACTIVE"] = data_frame.apply(lambda df: account_activity(df["ID"],
                                                                        df["LAST_RECORD"]), axis=1)
    
    data_frame["PARTIAL_STATUS"] = data_frame.apply(lambda df: status2number(df["MONTHS"],
                                                                             df["STATUS"],
                                                                             df["NUMBER_OF_RECORDS"],
                                                                             minimum_number_of_records,
                                                                             historical_range), axis=1)
    
    extra_frame = data_frame.groupby(by = "ID")["PARTIAL_STATUS"].sum().reset_index(name="TOTAL_STATUS")
    data_frame = pd.merge(data_frame, extra_frame, on="ID", how="left")
    
    data_frame["GOOD_BAD"] = data_frame.apply(lambda df: good_bad_function(df["ID"],
                                                                           df["TOTAL_STATUS"],
                                                                           threshold), axis=1)
    
    return data_frame

In [155]:
credit_records_new_df = main(credit_records, threshold = -0.4)

In [156]:
good_bad_dict_df = credit_records_new_df[["ID","GOOD_BAD"]].drop_duplicates().reset_index(drop = True)


In [157]:
good_bad_dict_df

,ID,GOOD_BAD
0,5001711,1
1,5001712,1
2,5001713,1
3,5001714,1
4,5001715,1
...,...,...
45980,5150482,1
45981,5150483,1
45982,5150484,1
45983,5150485,1


Join data frame good bad client with client data

In [158]:
final_df = pd.merge(application_records, good_bad_dict_df, how="left", on = "ID").reset_index(drop = True).drop ("Unnamed: 0", axis = 1)
final_df = final_df.drop("ID", axis = 1)

In [159]:
final_df

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,GOOD_BAD
0,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,0,0,Not provided,2,1.0
1,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,0,0,Not provided,2,1.0
2,M,1,1,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,0,0,0,Security staff,2,1.0
3,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,0,1,1,Sales staff,1,1.0
4,F,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,0,1,1,Sales staff,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,M,0,1,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,0,0,0,Not provided,1,NaN
438553,F,0,0,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,0,0,0,Laborers,1,NaN
438554,F,0,0,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,0,0,Sales staff,1,NaN
438555,F,0,1,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,0,0,0,Not provided,2,NaN


In [147]:
# import packages
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)

    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv) 

In [160]:
final_iv, IV = data_vars(final_df,final_df.GOOD_BAD)

In [161]:
final_iv[final_iv[["VAR_NAME", "WOE"]]["VAR_NAME"] == "OCCUPATION_TYPE"]

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
42,OCCUPATION_TYPE,Accountants,Accountants,1241,980.0,0.789686,261.0,0.210314,0.034650,0.031931,0.081730,0.010813
43,OCCUPATION_TYPE,Cleaning staff,Cleaning staff,551,404.0,0.733212,147.0,0.266788,0.014284,0.017984,-0.230320,0.010813
44,OCCUPATION_TYPE,Cooking staff,Cooking staff,655,493.0,0.752672,162.0,0.247328,0.017431,0.019819,-0.128390,0.010813
45,OCCUPATION_TYPE,Core staff,Core staff,3591,2780.0,0.774158,811.0,0.225842,0.098292,0.099217,-0.009364,0.010813
46,OCCUPATION_TYPE,Drivers,Drivers,2138,1652.0,0.772685,486.0,0.227315,0.058410,0.059457,-0.017769,0.010813
47,OCCUPATION_TYPE,HR staff,HR staff,85,54.0,0.635294,31.0,0.364706,0.001909,0.003793,-0.686306,0.010813
48,OCCUPATION_TYPE,High skill tech staff,High skill tech staff,1383,1071.0,0.774403,312.0,0.225597,0.037867,0.038170,-0.007958,0.010813
49,OCCUPATION_TYPE,IT staff,IT staff,60,45.0,0.750000,15.0,0.250000,0.001591,0.001835,-0.142690,0.010813
50,OCCUPATION_TYPE,Laborers,Laborers,6211,4939.0,0.795202,1272.0,0.204798,0.174628,0.155615,0.115270,0.010813
51,OCCUPATION_TYPE,Low-skill Laborers,Low-skill Laborers,175,140.0,0.800000,35.0,0.200000,0.004950,0.004282,0.144992,0.010813


In [142]:
IV.sort_values("IV")

,VAR_NAME,IV
2,CNT_FAM_MEMBERS,3.382345e-07
7,FLAG_OWN_CAR,1.616552e-05
3,CODE_GENDER,8.522377e-05
10,FLAG_WORK_PHONE,1.883961e-04
1,CNT_CHILDREN,2.017502e-04
6,FLAG_EMAIL,2.058594e-04
14,NAME_INCOME_TYPE,2.841471e-04
8,FLAG_OWN_REALTY,4.024961e-04
5,DAYS_EMPLOYED,4.721763e-04
9,FLAG_PHONE,6.270410e-04


In [146]:
IV[["VAR_NAME", "IV"]].sum()

VAR_NAME    AMT_INCOME_TOTALCNT_CHILDRENCNT_FAM_MEMBERSCOD...
IV                                                   0.016477
dtype: object